In [ ]:
# import the required packages
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, root_mean_squared_error
from sklearn.preprocessing import StandardScaler


# 1. Import and clean the data
data = pd.read_csv(r'F:\internship\winequalityN.csv')
df = data.copy()
df.dropna(axis=0, inplace=True)
df.drop(df.index[0:1460], axis=0, inplace=True)

# 2. Encode 'type' column
label = LabelEncoder()
df['type'] = label.fit_transform(df['type'])

# 3. Define features and target
# Note: We are using 4 features
features = ['volatile acidity', 'chlorides', 'density', 'alcohol']
x = df[features]
y = df['quality']


# 4. Split the data
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=42)

# 5. Scaling
scalar = StandardScaler()
# Fit and transform the training data
x_train_scale = scalar.fit_transform(x_train)
# IMPORTANT: Use .transform() only for test data to avoid data leakage
x_test_scale = scalar.transform(x_test)

# 6. Train the model
model = LinearRegression()
model.fit(x_train_scale, y_train)

# 7. Evaluate the model
y_pred = model.predict(x_test_scale)
r2score = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f'r2_score : {r2score*100:.2f}%')
print(f'mse : {mse:.4f}')
print(f'mae : {mae:.4f}')
print(f'rmse : {rmse:.4f}')

# 8. Predict the user given input
# FIX: Put all 4 values into a single 2D list [[va, cl, dens, alc]]
# The order must match your 'features' list exactly
user_input_raw = [[0.12,0.01,0.989,14.5]] 


# FIX: Scale the whole row at once using the already fitted scalar
input_value_scaled = scalar.transform(user_input_raw)

# FIX: Predict using the 2D scaled array
prediction = model.predict(input_value_scaled)


print(f'\nThe predicted wine quality for the given input is {prediction[0]*10:.1f} out of 10')



r2_score : 28.82%
mse : 0.5012
mae : 0.5467
rmse : 0.7080

The predicted wine quality for the given input is 7.0 out of 10


c:\Users\tk224\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
# 1. Load and Clean
data = pd.read_csv(r'F:\internship\winequalityN.csv')
df = data.dropna().copy()
# 2. Encode categorical data
label = LabelEncoder()
df['type'] = label.fit_transform(df['type'])

# 3. Use ALL features to provide maximum information
features = ['type', 'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 
            'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 
            'sulphates', 'alcohol']
x = df[features]
y = df['quality']
# 4. STEP ONE: Generate Polynomial Features (Degree 2)
# This creates interaction terms like alcohol * volatile acidity
poly = PolynomialFeatures(degree=2, include_bias=False)
x_poly = poly.fit_transform(x)
# 5. Split the NEW polynomial data
x_train, x_test, y_train, y_test = train_test_split(x_poly, y, train_size=0.8, random_state=42)
# 6. STEP TWO: Scaling (Crucial for Polynomial Regression)
scalar = StandardScaler()
x_train_scale = scalar.fit_transform(x_train)
x_test_scale = scalar.transform(x_test)

# 7. Train Linear Regression on the curved data
model = LinearRegression()
model.fit(x_train_scale, y_train)
# 8. Evaluation
y_pred = model.predict(x_test_scale)
r2score = r2_score(y_test, y_pred)
mse=mean_squared_error(y_test,y_pred)
# 1. Your raw input (12 features)
user_input_raw = [[
    1,      # type (White)
    6.8,    # fixed acidity
    0.10,   # VOLATILE ACIDITY (Minimum)
    0.45,   # CITRIC ACID (High/Fresh)
    1.1,    # residual sugar
    0.009,  # CHLORIDES (Minimum)
    35.0,   # free sulfur dioxide
    115.0,  # total sulfur dioxide
    0.988,  # DENSITY (Minimum)
    3.25,   # pH
    0.85,   # SULPHATES
    14.9    # ALCOHOL (Maximum)
]]
# 2. Transform to Polynomial (Turns 12 columns into 90)
input_poly = poly.transform(user_input_raw)
# 3. Scale the data (This stops the "exploding" -93.36 result)
input_scaled = scalar.transform(input_poly)
# 4. Predict
raw_prediction = model.predict(input_scaled)[0]
# 5. Manual "Clip" using Python's built-in functions (No Numpy)
# This ensures the score is at least 0 and at most 10
final_score = max(0, min(10, raw_prediction))
print(f'Predicted Wine Quality: {final_score:.2f}')
print(f"--- High Performance Linear Regression ---")
print(f'Enhanced R2 Score: {r2score:.2f}')
print(f'mse: {mse}')

Predicted Wine Quality: 8.05
--- High Performance Linear Regression ---
Enhanced R2 Score: 0.38
mse: 0.4882184475215206


c:\Users\tk224\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
